Python APIs with Requests 🐍🍬

<hr style="width:100%;height:10px;border-width:0;color:gray;background-color:DarkBlue">
<hr style="width:100%;height:10px;border-width:0;color:gray;background-color:MediumSeaGreen">

In [1]:
import pandas as pd
import requests
from datetime import date
from io import BytesIO

## REST APIs

One of the most common ways to make requests to a service on the web is using an API (*application programming interface*) setup with the [REST](https://realpython.com/api-integration-in-python/#rest-architecture) (*representational state transfer*) protocol. These provide URLs you can visit to perform certain actions and interact with the API, known as *endpoints*.

The two most common actions you can perform with REST APIs are **GET** to retrieve things, and **POST** to interact with things.

Let's look at GET first, and use a public API to play with it. We'll be using the Dog API, from [https://dog.ceo/dog-api/documentation/](https://dog.ceo/dog-api/documentation/)

In [8]:
# Send request with requests.get(), providing just a URL that the API should
# go to - this URL will just return a link to a random image of a dog!
# We save the response from our request into a response object

response = requests.get('https://dog.ceo/api/breeds/image/random')

Once we have our response saved, we can look at a number of things it contains. Often the most useful things to check first is just if our request worked or not. We can check this with `.status_code`, which contains the [HTTP response status code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) for our request. There are lot of these that can contain useful clues about how your request has gone, but the main one to remember is `200`, which means it worked okay.

In [9]:
# look at response code
print(response.status_code)

200


It can be useful to use this status code for control flow based on whether your query worked or not:

In [10]:
if response.status_code == 200:
    print("Success!")
else:
    print("Oh no!")

Success!


The main content our request returns to us is held in `.content`. By default, just calling this returns the content as a byte string, denoted by the `b` at the start of the string returned. This is pure raw data!

In [11]:
response.content

b'{"message":"https:\\/\\/images.dog.ceo\\/breeds\\/bouvier\\/n02106382_3692.jpg","status":"success"}'

This isn't always terribly useful though, so Requests also gives us some shortcuts to interpret this content as plain text, or some common data formats such as json:

In [12]:
# look at content of response with .text for plain text

response.text

'{"message":"https:\\/\\/images.dog.ceo\\/breeds\\/bouvier\\/n02106382_3692.jpg","status":"success"}'

In [14]:
# look at content of response, parsed as JSON

response.json()

{'message': 'https://images.dog.ceo/breeds/bouvier/n02106382_3692.jpg',
 'status': 'success'}

JSON is a very common method for transmitting data with APIs, and is very useful as it works with key:value pairs exactly the same as a regular Python dictionary. It's so common in fact, that Requests has a function built in to read content and parse it nicely as JSON for us! So we can use `.json()` to read our content, and can then save it as a dictionary and access elements within it using standard Python syntax.

In [25]:
# save the json to a dictionary

response_dict = response.json()
print(response_dict)

{'message': 'https://images.dog.ceo/breeds/bouvier/n02106382_3692.jpg', 'status': 'success'}


In [26]:
# access each part of our JSON response with square bracket syntax

response_dict['message']

'https://images.dog.ceo/breeds/bouvier/n02106382_3692.jpg'

In [18]:
# ... which in this case, lets us use our returned image URL to show us a dog!

from IPython.display import Image

Image(url = response_dict['message'])

### Headers

As well as the content of our response, it also contains some metadata referred to as 'headers'. There are lots of possible headers that you can send and receive with REST APIs, and they can be seen using `.headers`:

In [21]:
response.headers

{'Date': 'Mon, 10 Jul 2023 14:11:54 GMT', 'Content-Type': 'application/json', 'Content-Length': '106', 'Connection': 'keep-alive', 'X-Powered-By': 'PHP/8.1.0', 'Cache-Control': 'no-cache, private', 'Access-Control-Allow-Origin': '*', 'Content-Encoding': 'gzip', 'Via': '1.1 varnish (Varnish/6.3), 1.1 varnish (Varnish/6.3)', 'X-Cache-Hits': '0', 'X-Cache': 'MISS', 'Accept-Ranges': 'bytes', 'Age': '0', 'Vary': 'Accept-Encoding', 'CF-Cache-Status': 'DYNAMIC', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v3?s=26oV9317bENxHQqHdaKVICyUsVL1MIGtE5%2B3eOx0lSWcsvwgTCE75B1RXjFddYs%2BiqfQmHGItt7sO2b44nOxVUvJH6HZODK4Eshwy2Tki7SsaGIqJ56m7XMN"}],"group":"cf-nel","max_age":604800}', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'Server': 'cloudflare', 'CF-RAY': '7e49660a4d61749d-LHR', 'alt-svc': 'h3=":443"; ma=86400'}

In [22]:
# notice these headers are stored in a dictionary in our response object, so we can access certain parts of those as well:

response.headers['Date']

'Mon, 10 Jul 2023 14:11:54 GMT'

One of the main headers to be aware of is Content-Type, which tells us what type of data we are receiving from our API, often referred to as MIME type. There are lots of these, and you can see some common ones [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types). 

For this response, you can see Content-Type is telling us this is JSON data!

In [23]:
response.headers['Content-Type']

'application/json'

You can tell the API what type of data you want back using the `headers` argument, and sending an `Accept` header with a MIME type. Let's explicitly tell the server what data type we want back - often, if the server does not support a data type, you'll then get an error. This can be useful when APIs offer data in multiple formats, for example XML or JSON, and you want to pick one.

In [ ]:
requests.get('https://dog.ceo/api/breeds/image/random', headers = {'Accept': 'application/json'})

Let's play some more with the Dog API!

In [ ]:
# by sending a request to a different URL / endpoint, we can get different data back
# let's get a list of all the breeds available through Dog API

endpoint = 'https://dog.ceo/api/breeds/list/all'

breeds = requests.get(endpoint)

breeds = breeds.json()['message']

In [76]:
# we can loop through this now with a new endpoint we construct each time, to return a random image from each breed
# lets limit ourselves to just a random breed from the list to save time!

import random

# change k to another number to see more dogs at once!
random_breeds = random.choices(list(breeds.keys()), k = 1)

for breed in random_breeds:
    endpoint = f'https://dog.ceo/api/breed/{breed}/images/random'

    response = requests.get(endpoint)
    json = response.json()
    image_url = json['message']

    print(breed + ':')
    display(Image(url = image_url))

appenzeller:


### Queries

We can also interact with our APIs using queries that we add on to the end of our URL endpoints, usually be adding a ? and then our query. To see how this works, we can use `params` with our **GET** request to query the Agify API that guesses a person's age based solely on their name: [https://agify.io/](https://agify.io/)

In [13]:
# setup our endpoint URL as before

endpoint = 'https://api.agify.io'

# we can then make a request to this endpoint and use the 'params' argument to submit a name

response = requests.get(endpoint, params = {'name': 'James'})

In [ ]:
# PRACTISE: How can you view whether this request was successful or not?

In [ ]:
# PRACTISE: Look at the response to find the predicted age for this request

In [14]:
# we can provide multiple queries at once if the API supports it

queries = {'name': 'James Adams', 'country_id': 'GB'}

response = requests.get(endpoint, params = queries)

In [ ]:
# PRACTISE: Try look at the Agify API page to see what other queries you could add, and what difference they make to your results

In [15]:
# as part of our response object, we can see the entire URL query that was sent to our endpoint with .request.url

print(response.request.url)

# notice how our queries have been added on to our endpoint URL after a '?', and separated by '&'. We could have coded these by hand into our endpoint,
# but requests makes it easier and handles the URL encoding for us - otherwise we'd have to manually replace things like spaces with %20%!

https://api.agify.io/?name=James+Adams&country_id=GB


### POST

Unlike **GET**, we usually interact with **POST** by actually sending content in the *body* of our request, rather than by adding things on to the end of our URL. This can be done using the `data` argument to `requests.post()`, which can take data in a variety of formats but most often a Python dictionary.

In [7]:
# httpbin is a useful service that just echoes back any REST query you send to it, allowing you to test your queries

requests.post('https://httpbin.org/post', data = {'key': 'value'})

<Response [200]>

Commonly, the data we send is in JSON format, so that the server on the other end can read and interpret the request body easily. This is so common that, once again, Requests has it built in using the `json` argument in place of `data`. This can still accept a dictionary, but converts it into a JSON file to send to the URL endpoint for the server to use.

In [8]:
response = requests.post('https://httpbin.org/post', json = {'key': 'value'})

json_response = response.json()

json_response['data']

'{"key": "value"}'

You can send other types of data with Requests, such as files if you wanted to upload something to a server. Files need to be read into Python as raw binary data but then can be sent with the `files` argument. If we use the httpbin service to see how an uploaded file is represented, we can see that the file is converted into a very long string of text that encodes the actual file info.

In [ ]:
endpoint = 'https://httpbin.org/post'

files = {'file': open('mv-logo.png', 'rb')}

response = requests.post(endpoint, files = files)
response.text

We can also practise uploading things with **POST** using the [Pastebin API](https://pastebin.com/doc_api), to upload some text for us to share on the web. Before we do that though, we need to talk about authentication and APIs.

#### Authentication

Authentication can be a complicated topic in APIs. There are numerous ways you can authenticate yourself with a service that requires you to be logged in, from simply sending your username and password in a header (not recommended!) to setting up authorisation with services like OAuth (like Google). These vary from API service to service, and you will need to determine how the API wants you to provide your credentials. See [https://realpython.com/python-requests/#authentication](https://realpython.com/python-requests/#authentication) for some more info on some of these options.

One common method is to create API keys, or personal access tokens, to identify you as a user. These can be managed and, if need be, revoked without affecting your main login credentials, creating a more secure way to manage authorising yourself to an API.

### Getting a Pastebin API key

To sign up with Pastebin, head to: [https://pastebin.com/signup](https://pastebin.com/signup). Register for an account using any method you prefer.

Once logged in, head back to the API pages at [https://pastebin.com/doc_api](https://pastebin.com/doc_api) and you should see your API key near the top under the heading 'Your Unique Developer API Key'. Copy and paste this into the variable below!

In [2]:
PASTEBIN_API = 'Your API key here'

### Posting to Pastebin

Now we can construct a **POST** request to Pastebin to upload some text, and use our API key in a data field named `api_dev_key` to authenticate ourselves. Sometimes you will send your API key in `headers` instead, but for Pastebin, we just include it as part of the body of our request. Always read the API documentation to see how they want you to structure your requests!

In [3]:
endpoint = 'https://pastebin.com/api/api_post.php'

data = {'api_dev_key': PASTEBIN_API,
        'api_option': 'paste', # indicates we want to create a new pastebin entry
        'api_paste_code': 'Hello! This is using the pastebin API to upload this text.'} # the actual text we want to upload

response = requests.post(endpoint, data = data)

In [4]:
response.status_code

200

The response content is simply a link to our uploaded new text on Pastebin!

In [6]:
response.text

'https://pastebin.com/csLWE47j'

### API packages in Python

A lot of APIs, if they are popular enough, will already have a Python package available that will help you interact with them. Under the hood they are really just doing exactly the same thing we've been doing with Requests, but usually bundled up nicely to make it easier to work with an API. For example,  the [Dog API](https://pypi.org/project/pydogceo/) and [Agify](https://pypi.org/project/agify/) services we used earlier both have Python packages!

It's worth looking for a package that interacts with the API you are interested in using, as they often take a lot of the headaches out of things like authentication, and retrieving data in a useable format. But it's still useful to be able to use Requests, to understand what these packages are doing under the hood, and to make custom API requests for yourself if you need more than a package provides.